## tf-idf on the item description

tf-idf on the item description

In [1]:
import pandas as pd
import gzip

# Reading the review
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [2]:
# Reading the image features
import struct

def readImageFeatures(path):
    f = open(path, 'rb')
    while True:
        asin = f.read(10)
        if asin == '': break
        feature = []
        for i in range(4096):
            feature.append(struct.unpack('f', f.read(4)))
        yield asin, feature

In [3]:
%cd /Users/Deepthi/Documents/DSE/Capstone/Data/From Julius

/Users/Deepthi/Documents/DSE/Capstone/Data/From Julius


In [4]:
!ls

image_features_Clothing_Shoes_and_Jewelry.b
meta_Clothing_Shoes_and_Jewelry.json.gz
ratings_Clothing_Shoes_and_Jewelry.csv
review_Women.csv
review_Women.csv.gz
review_women_brand_updated.csv
review_women_brnd_fv_prce_updated.csv
review_women_brnd_fv_updated.csv
reviews_Clothing_Shoes_and_Jewelry.json.gz
reviews_Clothing_Shoes_and_Jewelry_5.json.gz
tfidf_score.csv


In [5]:
df_meta = getDF('meta_Clothing_Shoes_and_Jewelry.json.gz')

In [6]:
df_review = getDF('reviews_Clothing_Shoes_and_Jewelry_5.json.gz')

In [7]:
df_meta_brand = df_meta[['asin', 'title','brand']].drop_duplicates()

In [8]:
# df.id.str.split().str.get(0)

df_meta_brand['brand_0'] = df_meta_brand.title.str.split().str.get(0)
df_meta_brand['brand_1'] = df_meta_brand.title.str.split().str.get(1)

In [9]:
df_meta_brand['brand_combined'] = df_meta_brand['brand_0']+' '+df_meta_brand['brand_1']

In [10]:
df_meta_brand['brand_new']=df_meta_brand.brand.fillna(df_meta_brand.brand_combined)

In [11]:
df_meta_brand[['brand','brand_new']].head(25)

,brand,brand_new
0,Big Dreams,Big Dreams
1,Boutique Cutie,Boutique Cutie
2,NaN,SHINING IMAGE
3,NaN,RiZ Women's
4,NaN,Lantin White
5,NaN,NVC Unisex
6,NaN,Kismeth Eyewear
7,NaN,Max-MPH Black
8,NaN,FX1 Small
9,NaN,Riz Small


In [12]:
review_women = pd.read_csv('review_Women.csv')

In [13]:
review_women.head()

,reviewerID,asin,unixReviewTime,brand,price
0,A4KU0XJNBH674,B000051SEN,1372723200,Suunto,159.99
1,A3HY330W94JPUQ,B000051SEN,1354492800,Suunto,159.99
2,A1CHOKV10NEI8X,B000051SEN,1143936000,Suunto,159.99
3,AOUBIY0S651IU,B000051SEN,1399248000,Suunto,159.99
4,A1GPGBHBI6T2HJ,B000051SEN,1175558400,Suunto,159.99


In [14]:
df_meta_brand.head()

,asin,title,brand,brand_0,brand_1,brand_combined,brand_new
0,0000037214,Purple Sequin Tiny Dancer Tutu Ballet Dance Fa...,Big Dreams,Purple,Sequin,Purple Sequin,Big Dreams
1,0000031887,Ballet Dress-Up Fairy Tutu,Boutique Cutie,Ballet,Dress-Up,Ballet Dress-Up,Boutique Cutie
2,0123456479,SHINING IMAGE HUGE PINK LEATHER JEWELRY BOX / ...,NaN,SHINING,IMAGE,SHINING IMAGE,SHINING IMAGE
3,0456844570,RiZ Women's Beautify Crafted &frac12; Rimmed F...,NaN,RiZ,Women's,RiZ Women's,RiZ Women's
4,0456808574,Lantin White Visor Wrap Around Ski Style Aviat...,NaN,Lantin,White,Lantin White,Lantin White


In [15]:
review_women_brand_updated = review_women.merge(df_meta_brand, on = 'asin', how='left')[['reviewerID','asin','unixReviewTime','brand_new','price']]

In [16]:
review_women_brand_updated = review_women_brand_updated.rename(columns={'brand_new':'brand'})

In [17]:
# review_women_brand_updated.to_csv('review_women_brand_updated.csv')

In [18]:
# review_women_brand_updated.rename(columns={'brand_new':'brand'})

In [19]:
review_women_brand_updated

,reviewerID,asin,unixReviewTime,brand,price
0,A4KU0XJNBH674,B000051SEN,1372723200,Suunto,159.99
1,A3HY330W94JPUQ,B000051SEN,1354492800,Suunto,159.99
2,A1CHOKV10NEI8X,B000051SEN,1143936000,Suunto,159.99
3,AOUBIY0S651IU,B000051SEN,1399248000,Suunto,159.99
4,A1GPGBHBI6T2HJ,B000051SEN,1175558400,Suunto,159.99
5,A353P0P8WF325F,B000051SEN,1353628800,Suunto,159.99
6,A22E1XRKAQ2ZPP,B000051SEN,1383609600,Suunto,159.99
7,AM4NZRK0EFPU,B000051SEN,1239840000,Suunto,159.99
8,A2V98YBD3SRVG9,B00005KJXN,1385251200,Rubie&#39;s Costume Co,15.55
9,A2ZSX6R0TL8ET6,B00005KJXN,1368316800,Rubie&#39;s Costume Co,15.55


In [20]:
df_meta_only_womens = df_meta.merge(pd.DataFrame(review_women_brand_updated['asin'].drop_duplicates()), on='asin', how='inner')

In [21]:
product_title = list(df_meta_only_womens['title'].drop_duplicates().dropna())

In [22]:
## tf-idf example

from sklearn.feature_extraction.text import TfidfVectorizer
corpus = ["This is very strange",
          "This is very nice"]
vectorizer = TfidfVectorizer(min_df=1)
X = vectorizer.fit_transform(corpus)
idf = vectorizer.idf_
print dict(zip(vectorizer.get_feature_names(), idf))


{u'this': 1.0, u'very': 1.0, u'is': 1.0, u'strange': 1.4054651081081644, u'nice': 1.4054651081081644}


In [23]:
## Use tfidf on the title

import sklearn.feature_extraction.text 

my_stop_words = sklearn.feature_extraction.text.ENGLISH_STOP_WORDS.union({'women','womens'})

corpus = product_title
vectorizer = TfidfVectorizer(stop_words = set(my_stop_words),ngram_range=(2,2))
X = vectorizer.fit_transform(corpus)
idf = vectorizer.idf_
print dict(zip(vectorizer.get_feature_names(), idf))


{u'cat print': 9.8977506638182167, u'takes hand': 9.8977506638182167, u'la ray': 9.8977506638182167, u'body message': 9.8977506638182167, u'chess military': 9.8977506638182167, u'spandex fabric': 9.8977506638182167, u'hanes shapers': 9.8977506638182167, u'rosary faceted': 9.8977506638182167, u'ipanema maya': 9.8977506638182167, u'smocked tube': 9.8977506638182167, u'hooper tall': 9.8977506638182167, u'belly bar': 8.9814599319440624, u'tri mountain': 8.7991383751501075, u'steel tanzanite': 9.8977506638182167, u'world happy': 9.8977506638182167, u'floral animal': 9.8977506638182167, u'design pashmina': 9.2046034832582713, u'fit supersoft': 9.8977506638182167, u'make frisky': 9.8977506638182167, u'98d107 marine': 9.8977506638182167, u'circulatory crew': 9.8977506638182167, u'workout sneaker': 9.8977506638182167, u'wither bead': 9.8977506638182167, u'pattern brief': 9.8977506638182167, u'waisted cropped': 9.8977506638182167, u'white wings': 9.8977506638182167, u'collar pack': 9.89775066381

In [24]:
tf_idf_scores = pd.DataFrame(dict(zip(vectorizer.get_feature_names(), idf)).items())


In [25]:
tf_idf_scores.columns = ['word','tf_idf_score']

In [26]:
## tf_idf_scores.sort('tf_idf_score').to_csv('tfidf_score.csv')

In [27]:
two_grams = list(tf_idf_scores.sort('tf_idf_score').head(800)['word'])

/Users/Deepthi/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [28]:
# two_grams[0]

# df_meta_only_womens[]

In [29]:
two_grams[0]

u'sterling silver'

In [30]:
for i in range(len(two_grams)):
    print i
    df_meta_only_womens[two_grams[i]]=df_meta_only_womens['title'].fillna('0').str.lower().str.contains(two_grams[i]).astype(int)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [31]:
df_meta_only_womens['feature_vector'] = df_meta_only_womens[two_grams[0]].astype(str)

for i in range(1,len(two_grams)):
    df_meta_only_womens['feature_vector'] = df_meta_only_womens['feature_vector'].astype(str)\
                                        +df_meta_only_womens[two_grams[i]].astype(str)

# df_meta_only_womens['feature_vector']=df_meta_only_womens[0].str.cat(df_meta_only_womens[1])

In [32]:
df_meta_only_womens_with_fvectors = df_meta_only_womens[['asin','feature_vector']]

In [33]:
df_meta_only_womens_with_fvectors.head()

,asin,feature_vector
0,B000051SEN,0000000000000000000000000000000000000000000000...
1,B00005KJXN,0000000000000000000000000000000000000000000000...
2,B000074RL3,0000000000000000000000000000000000000000000000...
3,B000078CYM,0000000000000000000000000000000000000000000000...
4,B00007FFL9,0000000000000000000000000000000000000000000000...


In [34]:
review_women_brand_updated = pd.read_csv('review_women_brand_updated.csv')
##review_women_brand_updated=review_women_brand_updated.drop('Unnamed: 0',axis=1)

In [35]:
review_women_brnd_fv_updated=review_women_brand_updated.merge(df_meta_only_womens_with_fvectors, on='asin',how='left')

In [36]:
## review_women_brnd_fv_updated.to_csv('review_women_brnd_fv_updated.csv',index=False)

In [37]:
review_women_brnd_fv_updated.head()

,reviewerID,asin,unixReviewTime,brand,price,feature_vector
0,A4KU0XJNBH674,B000051SEN,1372723200,Suunto,159.99,0000000000000000000000000000000000000000000000...
1,A3HY330W94JPUQ,B000051SEN,1354492800,Suunto,159.99,0000000000000000000000000000000000000000000000...
2,A1CHOKV10NEI8X,B000051SEN,1143936000,Suunto,159.99,0000000000000000000000000000000000000000000000...
3,AOUBIY0S651IU,B000051SEN,1399248000,Suunto,159.99,0000000000000000000000000000000000000000000000...
4,A1GPGBHBI6T2HJ,B000051SEN,1175558400,Suunto,159.99,0000000000000000000000000000000000000000000000...


## Clustering

In [38]:
from sklearn.cluster import KMeans
import numpy as np

fv_np_array = np.array(df_meta_only_womens_with_fvectors['feature_vector'])

fv_np_array2 = []
for i in range(len(fv_np_array)):
    fv_np_array2.append(list(fv_np_array[i]))

In [39]:
len(fv_np_array2)

14780

In [40]:
from sklearn.cluster import KMeans

In [41]:
kmeans = KMeans(n_clusters=800, random_state=0).fit(fv_np_array2)

In [42]:
kmeans.labels_

array([ 0,  0, 40, ...,  0,  0,  0], dtype=int32)

In [43]:
kmeans.predict(fv_np_array2)

array([ 0,  0, 40, ...,  0,  0,  0], dtype=int32)

In [44]:
# kmeans.cluster_centers_

# set(kmeans.predict(fv_np_array2))

In [45]:
clusters=pd.DataFrame(kmeans.predict(fv_np_array2))

In [46]:
clusters.columns=['clusters']

In [47]:
df_meta_only_womens_clusters = df_meta_only_womens.join(clusters)

In [48]:
df_meta_only_womens_clusters.groupby('clusters').count()

,asin,categories,title,price,salesRank,imUrl,brand,related,description,sterling silver,...,circle pendant,cole haan,silver 5mm,white large,mock wrap,hand ring,comfort bra,gold diamond,fashion corset,feature_vector
clusters,,,,,,,,,,,,,,,,,,,,,
0,5750,5750,5745,2078,5501,5750,759,5722,246,5750,...,5750,5750,5750,5750,5750,5750,5750,5750,5750,5750
1,6,6,6,5,6,6,0,6,0,6,...,6,6,6,6,6,6,6,6,6,6
2,119,119,119,4,118,119,0,119,0,119,...,119,119,119,119,119,119,119,119,119,119
3,17,17,17,15,17,17,0,16,0,17,...,17,17,17,17,17,17,17,17,17,17
4,51,51,51,36,51,51,2,51,0,51,...,51,51,51,51,51,51,51,51,51,51
5,84,84,84,10,84,84,0,83,0,84,...,84,84,84,84,84,84,84,84,84,84
6,56,56,56,47,55,56,15,55,0,56,...,56,56,56,56,56,56,56,56,56,56
7,156,156,156,60,153,156,7,156,7,156,...,156,156,156,156,156,156,156,156,156,156
8,106,106,106,12,105,106,1,106,0,106,...,106,106,106,106,106,106,106,106,106,106


In [49]:
### Columbia Sportswear Women's Heather Canyon Wp: price is #999 in the dataset. But it is actually 99.99 so updating that

# df_meta_only_womens_clusters[df_meta_only_womens_clusters['asin']=='B004LP2OJU']

df_meta_only_womens_clusters.set_value(5404, 'price', 99.99)

,asin,categories,title,price,salesRank,imUrl,brand,related,description,sterling silver,...,cole haan,silver 5mm,white large,mock wrap,hand ring,comfort bra,gold diamond,fashion corset,feature_vector,clusters
0,B000051SEN,"[[Electronics, GPS & Navigation, Sports & Hand...","Suunto Vector Wristop Computer with Compass, A...",159.99,{u'Watches': 1647},http://ecx.images-amazon.com/images/I/51MH7ZB4...,Suunto,"{u'also_bought': [u'B00005NZWK', u'B000IU6RN6'...",NaN,0,...,0,0,0,0,0,0,0,0,0000000000000000000000000000000000000000000000...,0
1,B00005KJXN,"[[Clothing, Shoes & Jewelry, Novelty, Costumes...",Rubies Costume Co Women's FourInOne Nun Costume,15.55,{u'Clothing': 37747},http://ecx.images-amazon.com/images/I/31ZJCQD5...,Rubie&#39;s Costume Co,"{u'also_bought': [u'B003QHZAO0', u'B000PLTLNG'...",NaN,0,...,0,0,0,0,0,0,0,0,0000000000000000000000000000000000000000000000...,0
2,B000074RL3,"[[Clothing, Shoes & Jewelry, Novelty, Costumes...",Enell High Impact Sports Bra,NaN,{u'Sports &amp; Outdoors': 5895},http://ecx.images-amazon.com/images/I/411%2BzN...,NaN,"{u'also_bought': [u'B001BDV9SC', u'B0035WTMF4'...",The Enell High Impact Sports Bra &lt;span&gt;i...,0,...,0,0,0,0,0,0,0,0,0000000000000000000000000000000000000000000000...,40
3,B000078CYM,"[[Clothing, Shoes & Jewelry, Shoes & Accessori...",AmeriBag X-Small Distressed Nylon Healthy Back...,49.21,{u'Clothing': 27502},http://ecx.images-amazon.com/images/I/41ypKZPn...,AmeriBag,"{u'also_bought': [u'B001CE64C6', u'B001CE49T6'...",NaN,0,...,0,0,0,0,0,0,0,0,0000000000000000000000000000000000000000000000...,0
4,B00007FFL9,"[[Clothing, Shoes & Jewelry, Women], [Clothing...",PalmBeach Jewelry Open Weave Decorative Band i...,NaN,{u'Jewelry': 76973},http://ecx.images-amazon.com/images/I/51EvJUOB...,NaN,"{u'also_bought': [u'B000X2DRRI', u'B00BRCYT98'...",NaN,0,...,0,0,0,0,0,0,0,0,0000000000000000000000000000000000000000000000...,191
5,B00007GDD3,"[[Clothing, Shoes & Jewelry, Women, Handbags &...",Buxton Double-Flap Mini-Bag with Total Wallet ...,18.37,NaN,http://ecx.images-amazon.com/images/I/41EHat-F...,Buxton,"{u'also_bought': [u'B003E76ME4', u'B0048991OU'...",NaN,0,...,0,0,0,0,0,0,0,0,0000000000000000000000000000000000000000000000...,0
6,B00007GDAL,"[[Clothing, Shoes & Jewelry, Women, Accessorie...",Buxton Heiress Double CardexTM,19.99,NaN,http://ecx.images-amazon.com/images/I/41ENA0kC...,Buxton,"{u'also_bought': [u'B00021Q2JU', u'B008E7TQYC'...",NaN,0,...,0,0,0,0,0,0,0,0,0000000000000000000000000000000000000000000000...,0
7,B00007GD9I,"[[Clothing, Shoes & Jewelry, Women, Handbags &...",Buxton Heiress Organizer; Clutch,26.47,NaN,http://ecx.images-amazon.com/images/I/41cga2O2...,Buxton,"{u'also_bought': [u'B00JXRBM1C', u'B00CYR6EX4'...",NaN,0,...,0,0,0,0,0,0,0,0,0000000000000000000000000000000000000000000000...,0
8,B00007GD8X,"[[Clothing, Shoes & Jewelry, Women, Handbags &...",Buxton Check Clutch Mini Bag On A String,13.64,NaN,http://ecx.images-amazon.com/images/I/41oJPUrC...,Buxton,"{u'also_bought': [u'B004C51NRS', u'B003E76ME4'...",NaN,0,...,0,0,0,0,0,0,0,0,0000000000000000000000000000000000000000000000...,0
9,B00007GD9W,"[[Clothing, Shoes & Jewelry, Women, Accessorie...",Buxton Heiress Convertible&reg; Billfold,18.02,{u'Clothing': 28110},http://ecx.images-amazon.com/images/I/41olO1DM...,Buxton,"{u'also_bought': [u'B00523T0TC', u'B000E83VU4'...",NaN,0,...,0,0,0,0,0,0,0,0,0000000000000000000000000000000000000000000000...,0


In [50]:
df_meta_only_womens_clusters[df_meta_only_womens_clusters['clusters']==16]

,asin,categories,title,price,salesRank,imUrl,brand,related,description,sterling silver,...,cole haan,silver 5mm,white large,mock wrap,hand ring,comfort bra,gold diamond,fashion corset,feature_vector,clusters
216,B00061RG1O,"[[Clothing, Shoes & Jewelry, Women, Jewelry, F...",Brilliant Cut Clear Cubic Zirconia Solitaire 3...,20.99,{u'Jewelry': 47493},http://ecx.images-amazon.com/images/I/31s5Zew2...,NaN,"{u'also_bought': [u'B000OC27OQ', u'B006OHTI24'...",NaN,1,...,0,0,0,0,0,0,0,0,1000010000000000000000000000000000010000000000...,16
907,B000KL20GG,"[[Clothing, Shoes & Jewelry, Women, Jewelry, F...",Sterling Silver Womens Band Princess Cut Cubic...,20.99,{u'Jewelry': 30375},http://ecx.images-amazon.com/images/I/315BRk74...,NaN,"{u'also_bought': [u'B006OHTI24', u'B000OC27OQ'...",NaN,1,...,0,0,0,0,0,0,0,0,1000010000000000000000000000000000010000000000...,16
4531,B0040LII7M,"[[Clothing, Shoes & Jewelry, Women], [Clothing...",Sterling Silver Cubic Zirconia Solitaire Stud ...,NaN,{u'Jewelry': 56612},http://ecx.images-amazon.com/images/I/41pI9r5c...,NaN,"{u'also_bought': [u'B004Q7AB4I', u'B004OPZ3J0'...",NaN,1,...,0,0,0,0,0,0,0,0,1000010100000000000000000000000000000000000000...,16
5111,B004GXBP1U,"[[Clothing, Shoes & Jewelry, Jewelry: Internat...",Sterling Silver Bezel Martini Set Cubic Zircon...,12.10,{u'Jewelry': 979},http://ecx.images-amazon.com/images/I/41W9l7U2...,NaN,"{u'also_bought': [u'B003Y5E964', u'B004JZWX52'...",NaN,1,...,0,0,0,0,0,0,0,0,1000010000000000000000000000000000000000000000...,16
12274,B00B4QKPSQ,"[[Clothing, Shoes & Jewelry, Women], [Clothing...",FM42 Gold Tone Diamond-cut Cubic Zirconia Soli...,2.98,{u'Jewelry': 3492},http://ecx.images-amazon.com/images/I/41NfVFc2...,NaN,"{u'also_bought': [u'B00B3S3BH2', u'B00BRJ2182'...",NaN,0,...,0,0,0,0,0,0,0,0,0000010000000000000000000000000000000100000000...,16


In [51]:
df_cluster_mean_price = pd.DataFrame(df_meta_only_womens_clusters[df_meta_only_womens_clusters['price'].notnull()]\
                        .groupby('clusters').price.mean()).reset_index()

df_cluster_mean_price.columns=['clusters','mean_price']
# .to_csv('validate_price.csv')

In [52]:
# pd.DataFrame(df_cluster_mean_price).reset_index()

## Merge the mean price back to original data

# df_cluster_mean_price

df_meta_only_womens_clusters = df_meta_only_womens_clusters.merge(df_cluster_mean_price,on='clusters')

In [53]:
df_meta_only_womens_clusters

,asin,categories,title,price,salesRank,imUrl,brand,related,description,sterling silver,...,silver 5mm,white large,mock wrap,hand ring,comfort bra,gold diamond,fashion corset,feature_vector,clusters,mean_price
0,B000051SEN,"[[Electronics, GPS & Navigation, Sports & Hand...","Suunto Vector Wristop Computer with Compass, A...",159.99,{u'Watches': 1647},http://ecx.images-amazon.com/images/I/51MH7ZB4...,Suunto,"{u'also_bought': [u'B00005NZWK', u'B000IU6RN6'...",NaN,0,...,0,0,0,0,0,0,0,0000000000000000000000000000000000000000000000...,0,26.327955
1,B00005KJXN,"[[Clothing, Shoes & Jewelry, Novelty, Costumes...",Rubies Costume Co Women's FourInOne Nun Costume,15.55,{u'Clothing': 37747},http://ecx.images-amazon.com/images/I/31ZJCQD5...,Rubie&#39;s Costume Co,"{u'also_bought': [u'B003QHZAO0', u'B000PLTLNG'...",NaN,0,...,0,0,0,0,0,0,0,0000000000000000000000000000000000000000000000...,0,26.327955
2,B000078CYM,"[[Clothing, Shoes & Jewelry, Shoes & Accessori...",AmeriBag X-Small Distressed Nylon Healthy Back...,49.21,{u'Clothing': 27502},http://ecx.images-amazon.com/images/I/41ypKZPn...,AmeriBag,"{u'also_bought': [u'B001CE64C6', u'B001CE49T6'...",NaN,0,...,0,0,0,0,0,0,0,0000000000000000000000000000000000000000000000...,0,26.327955
3,B00007GDD3,"[[Clothing, Shoes & Jewelry, Women, Handbags &...",Buxton Double-Flap Mini-Bag with Total Wallet ...,18.37,NaN,http://ecx.images-amazon.com/images/I/41EHat-F...,Buxton,"{u'also_bought': [u'B003E76ME4', u'B0048991OU'...",NaN,0,...,0,0,0,0,0,0,0,0000000000000000000000000000000000000000000000...,0,26.327955
4,B00007GDAL,"[[Clothing, Shoes & Jewelry, Women, Accessorie...",Buxton Heiress Double CardexTM,19.99,NaN,http://ecx.images-amazon.com/images/I/41ENA0kC...,Buxton,"{u'also_bought': [u'B00021Q2JU', u'B008E7TQYC'...",NaN,0,...,0,0,0,0,0,0,0,0000000000000000000000000000000000000000000000...,0,26.327955
5,B00007GD9I,"[[Clothing, Shoes & Jewelry, Women, Handbags &...",Buxton Heiress Organizer; Clutch,26.47,NaN,http://ecx.images-amazon.com/images/I/41cga2O2...,Buxton,"{u'also_bought': [u'B00JXRBM1C', u'B00CYR6EX4'...",NaN,0,...,0,0,0,0,0,0,0,0000000000000000000000000000000000000000000000...,0,26.327955
6,B00007GD8X,"[[Clothing, Shoes & Jewelry, Women, Handbags &...",Buxton Check Clutch Mini Bag On A String,13.64,NaN,http://ecx.images-amazon.com/images/I/41oJPUrC...,Buxton,"{u'also_bought': [u'B004C51NRS', u'B003E76ME4'...",NaN,0,...,0,0,0,0,0,0,0,0000000000000000000000000000000000000000000000...,0,26.327955
7,B00007GD9W,"[[Clothing, Shoes & Jewelry, Women, Accessorie...",Buxton Heiress Convertible&reg; Billfold,18.02,{u'Clothing': 28110},http://ecx.images-amazon.com/images/I/41olO1DM...,Buxton,"{u'also_bought': [u'B00523T0TC', u'B000E83VU4'...",NaN,0,...,0,0,0,0,0,0,0,0000000000000000000000000000000000000000000000...,0,26.327955
8,B00007GDG5,"[[Clothing, Shoes & Jewelry, Women, Accessorie...",Buxton Heiress Ensemble Clutch,19.52,NaN,http://ecx.images-amazon.com/images/I/41FaRvaT...,Buxton,"{u'also_bought': [u'B00GQJ5AL8', u'B001O1V2ZG'...",NaN,0,...,0,0,0,0,0,0,0,0000000000000000000000000000000000000000000000...,0,26.327955
9,B00007IVVR,"[[Clothing, Shoes & Jewelry, Women, Petite, In...",Braza Adhesive Bra,NaN,{u'Clothing': 2989},http://ecx.images-amazon.com/images/I/418z8y06...,NaN,"{u'also_bought': [u'B00008IOAE', u'B005LGV8HW'...",NaN,0,...,0,0,0,0,0,0,0,0000000000000000000000000000000000000000000000...,0,26.327955


In [54]:
df_meta_only_womens_clusters['price_new']=df_meta_only_womens_clusters.price\
                                          .fillna(df_meta_only_womens_clusters.mean_price)

In [55]:
df_updated_price = df_meta_only_womens_clusters[['asin','price_new']].drop_duplicates()

In [56]:
# df_meta_only_womens_clusters[df_meta_only_womens_clusters['asin']=='B004LP2OJU']['title']

In [57]:
review_women_brnd_fv_prce_updated = review_women_brnd_fv_updated.merge(df_updated_price,on='asin',how='left')\
                                   [['reviewerID','asin','unixReviewTime','brand','price_new','feature_vector']]

In [58]:
review_women_brnd_fv_prce_updated.columns=['reviewerID','asin','unixReviewTime','brand','price','feature_vector']

In [59]:
import string
import re

RE_PUNCTUATION = '|'.join([re.escape(x) for x in string.punctuation])  

review_women_brnd_fv_prce_updated['brand']=review_women_brnd_fv_prce_updated['brand'].str.lower()\
                                            .str.replace(RE_PUNCTUATION, " ")

# review_women_brnd_fv_prce_updated['brand'].str.lower()\
# .apply(lambda x:''.join([i for i in x if i not in string.punctuation]))

In [60]:
review_women_brnd_fv_prce_updated.to_csv('review_women_brnd_fv_prce_updated.csv',index=False)

In [61]:
# review_women_brnd_fv_prce_updated

,reviewerID,asin,unixReviewTime,brand,price,feature_vector
0,A4KU0XJNBH674,B000051SEN,1372723200,suunto,159.990000,0000000000000000000000000000000000000000000000...
1,A3HY330W94JPUQ,B000051SEN,1354492800,suunto,159.990000,0000000000000000000000000000000000000000000000...
2,A1CHOKV10NEI8X,B000051SEN,1143936000,suunto,159.990000,0000000000000000000000000000000000000000000000...
3,AOUBIY0S651IU,B000051SEN,1399248000,suunto,159.990000,0000000000000000000000000000000000000000000000...
4,A1GPGBHBI6T2HJ,B000051SEN,1175558400,suunto,159.990000,0000000000000000000000000000000000000000000000...
5,A353P0P8WF325F,B000051SEN,1353628800,suunto,159.990000,0000000000000000000000000000000000000000000000...
6,A22E1XRKAQ2ZPP,B000051SEN,1383609600,suunto,159.990000,0000000000000000000000000000000000000000000000...
7,AM4NZRK0EFPU,B000051SEN,1239840000,suunto,159.990000,0000000000000000000000000000000000000000000000...
8,A2V98YBD3SRVG9,B00005KJXN,1385251200,rubie 39 s costume co,15.550000,0000000000000000000000000000000000000000000000...
9,A2ZSX6R0TL8ET6,B00005KJXN,1368316800,rubie 39 s costume co,15.550000,0000000000000000000000000000000000000000000000...
